安装依赖

In [1]:
# !pip install mindspore==2.3.1 mindnlp==0.4.1
!pip install datasets emoji scikit-learn

Looking in indexes: http://mirrors.aliyun.com/pypi/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 590.6/590.6 kB 30.2 MB/s eta 0:00:00
DEPRECATION: moxing-framework 2.1.16.2ae09d45 has a non-standard version number. pip 24.0 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of moxing-framework or contact the author to suggest that they release a version with a conforming version number. Discussion can be found at https://github.com/pypa/pip/issues/12063


In [2]:
import mindspore
mindspore.set_context(device_target='Ascend', device_id=0, pynative_synchronize=True)

加载数据集

In [ ]:
from datasets import load_dataset

dataset = load_dataset("/tmp/code/hate_speech_twitter")
# dataset = load_dataset("thefrankhsu/hate_speech_twitter")
train_dataset = dataset['train'].remove_columns('categories').filter(lambda x: x['tweet'] is not None and x['label'] in [0, 1])
test_dataset = dataset['test'].remove_columns('categories').filter(lambda x: x['tweet'] is not None and x['label'] in [0, 1])

print("train dataset num_rows: ", train_dataset.num_rows)
print("test dataset num_rows: ", test_dataset.num_rows)
print(train_dataset.with_format("pandas")[:5])

/home/ma-user/anaconda3/envs/MindSpore/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Generating train split: 5679 examples [00:00, 145319.43 examples/s]
Generating test split: 1000 examples [00:00, 72245.83 examples/s]
Filter: 100%|██████████| 1000/1000 [00:00<00:00, 68059.52 examples/s]

train dataset num_rows:  5678
test dataset num_rows:  1000
                                               tweet  label
0  krazy i dont always get drunk and pass out but...      0
1  white kids favorite activities calling people ...      1
2  maam did you clear that tweet with the   caref...      0
3  wth is that playing missy  i mean seriously rt...      0
4           he promised to stand with the muzzies so      0


加载模型

In [4]:
from mindnlp.transformers import BertweetTokenizer, AutoModelForSequenceClassification

tokenizer = BertweetTokenizer.from_pretrained("/tmp/code/bertweet-base")
# tokenizer = BertweetTokenizer.from_pretrained("vinai/bertweet-base")
model = AutoModelForSequenceClassification.from_pretrained("/tmp/code/bertweet-base", num_labels=2)
# model = AutoModelForSequenceClassification.from_pretrained("vinai/bertweet-base", num_labels=2)

Building prefix dict from the default dictionary ...
Dumping model to file cache /tmp/jieba.cache
Loading model cost 1.412 seconds.
Prefix dict has been built successfully.
/home/ma-user/anaconda3/envs/MindSpore/lib/python3.9/site-packages/mindnlp/transformers/tokenization_utils_base.py:1526: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted, and will be then set to `False` by default. 
  warnings.warn(


[MS_ALLOC_CONF]Runtime config:  enable_vmm:True  vmm_align_size:2MB


Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at /tmp/code/bertweet-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


数据预处理

In [ ]:
import mindspore
from mindspore.dataset import GeneratorDataset, transforms

class HSTDataset:
    def __init__(self, dataset):
        self.dataset = dataset

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, idx):
        if not isinstance(idx, int):
            idx = int(idx)
        return self.dataset[idx]['tweet'], self.dataset[idx]['label']

def process_dataset(source, tokenizer, max_seq_len=64, batch_size=32, shuffle=True, drop_remainder=False):
    is_ascend = mindspore.get_context('device_target') == 'Ascend'

    column_names = ["tweet", "label"]

    dataset = GeneratorDataset(source, column_names=column_names, shuffle=shuffle)
    # transforms
    type_cast_op = transforms.TypeCast(mindspore.int32)
    def tokenize_and_pad(text):
        if is_ascend:
            tokenized = tokenizer(text, padding='max_length', truncation=True, max_length=max_seq_len)
        else:
            tokenized = tokenizer(text)
        return tokenized['input_ids'], tokenized['attention_mask']
    # map dataset
    dataset = dataset.map(operations=tokenize_and_pad, input_columns="tweet", output_columns=['input_ids', 'attention_mask'])
    dataset = dataset.map(operations=[type_cast_op], input_columns="label", output_columns='labels')
    # # batch dataset
    if is_ascend:
        dataset = dataset.batch(batch_size, drop_remainder=drop_remainder)
    else:
        dataset = dataset.padded_batch(batch_size, drop_remainder=drop_remainder, 
                                       pad_info={'input_ids': (None, tokenizer.pad_token_id),
                                                'attention_mask': (None, 0)})

    return dataset

train_dataset = process_dataset(HSTDataset(train_dataset), tokenizer, drop_remainder=True)
test_dataset = process_dataset(HSTDataset(test_dataset), tokenizer, shuffle=False)

In [6]:
print(next(train_dataset.create_dict_iterator()))

{'input_ids': Tensor(shape=[32, 64], dtype=Int64, value=
[[    0,    37,  1484 ...     1,     1,     1],
 [    0,   462, 10898 ...     1,     1,     1],
 [    0,   111,   112 ...     1,     1,     1],
 ...
 [    0,     6,    83 ...     1,     1,     1],
 [    0,   322,  1472 ...     1,     1,     1],
 [    0,   460, 51959 ...     1,     1,     1]]), 'attention_mask': Tensor(shape=[32, 64], dtype=Int64, value=
[[1, 1, 1 ... 0, 0, 0],
 [1, 1, 1 ... 0, 0, 0],
 [1, 1, 1 ... 0, 0, 0],
 ...
 [1, 1, 1 ... 0, 0, 0],
 [1, 1, 1 ... 0, 0, 0],
 [1, 1, 1 ... 0, 0, 0]]), 'labels': Tensor(shape=[32], dtype=Int32, value= [1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1, 0, 
 0, 0, 0, 1, 0, 0, 0, 0])}


评估函数

In [7]:
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
import numpy as np

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)

    accuracy = accuracy_score(labels, predictions)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, predictions, average='binary')

    return {
        'accuracy': accuracy,
        'precision': precision,
        'recall': recall,
        'f1': f1
    }

设置训练参数

In [8]:
from mindnlp.engine import TrainingArguments

training_args = TrainingArguments(
    output_dir="./bertweet_finetune/trainer_output",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    save_total_limit=2,
    load_best_model_at_end=True,
    metric_for_best_model='accuracy',
    greater_is_better=True,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    num_train_epochs=5,
    learning_rate=2e-5,
    weight_decay=0.01,
    logging_strategy="epoch"
)

加载Trainer

In [9]:
from mindnlp.engine import Trainer

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

训练

In [10]:
# Train the model
trainer.train()

print("Train over!")

  0%|          | 0/885 [00:00<?, ?it/s]

 20%|██        | 177/885 [03:14<10:29,  1.12it/s]

 20%|██        | 177/885 [03:15<10:29,  1.12it/s]

{'loss': 0.3052, 'learning_rate': 1.6000000000000003e-05, 'epoch': 1.0}



 97%|█████████▋| 31/32 [00:04<00:00,  7.96it/s]
                                                 A
100%|██████████| 32/32 [00:04<00:00,  7.74it/s]
                                               

...

{'loss': 0.0528, 'learning_rate': 0.0, 'epoch': 5.0}



 97%|█████████▋| 31/32 [00:05<00:00,  7.46it/s]
                                                 A
100%|██████████| 32/32 [00:05<00:00,  7.50it/s]
                                               

{'eval_loss': 0.9358894228935242, 'eval_accuracy': 0.77, 'eval_precision': 0.944078947368421, 'eval_recall': 0.574, 'eval_f1': 0.7139303482587064, 'eval_runtime': 5.9235, 'eval_samples_per_second': 5.402, 'eval_steps_per_second': 0.169, 'epoch': 5.0}


100%|██████████| 885/885 [16:17<00:00,  1.10s/it]

{'train_runtime': 977.8286, 'train_samples_per_second': 28.962, 'train_steps_per_second': 0.905, 'train_loss': 0.13215566516596045, 'epoch': 5.0}
Train over!


In [12]:
from mindspore import Tensor
def predict(text, label=None):
    label_map = {0: "Non Hate Speech", 1: "Hate Speech"}

    text_tokenized = Tensor([tokenizer(text).input_ids])
    logits = model(text_tokenized)
    predict_label = logits[0].asnumpy().argmax()
    info = f"inputs: '{text}', predict: '{label_map[predict_label]}'"
    if label is not None:
        info += f" , label: '{label_map[label]}'"
    print(info)
predict("on my way to fuck your bitch in the name of The Lord")

inputs: 'on my way to fuck your bitch in the name of The Lord', predict: 'Hate Speech'
